In [ ]:
!pip install --quiet pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
#First, we mount the google drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
#importing the math library
import math

In [ ]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("word count").setMaster("local")
sc = SparkContext.getOrCreate(conf=conf)

1. Project and print (empno, name) of employees that are from state 'TX' from employee.csv

In [ ]:
data = sc.textFile("/content/gdrive/My Drive/employee.csv")

employee = data.map(lambda data: data.split(','))

employee_TX = employee.filter(lambda e: str(e[4])=="TX")

for value in employee_TX.collect():
  print(value[0] + " "+value[1])

10012 Hector Barbossa
10200 Anton Chigurh
10202 Jac McKinzie


2. Generate List of (empno, name, salary, dep_avg_sal) of employees who have salary > 1.5 times of department average from employee.csv

In [ ]:
data = sc.textFile("/content/gdrive/My Drive/employee.csv")

employee = data.map(lambda data: data.split(','))

employee_salary = employee.map(lambda record: (record[2] , (int(record[3]),1)))

department_salary = employee_salary.reduceByKey(lambda S1,S2 : (S1[0] + S2[0] , S1[1]+S2[1]))

average_dno = department_salary.map(lambda S:(S[0],round(1.0*S[1][0]/S[1][1],2)))

Dict = {}
#defining a dictionary
for i in average_dno.collect():
  Dict[int(i[0])] = i[1]

employee2 = employee.filter(lambda record : float(record[3]) > (1.5*Dict[int(record[2])]) )

print("EmployeeNo | Name | Salary | Average Salary of Department")
for emp in employee2.collect():
  print(emp[0] + " "+ emp[1]+" "+emp[3]+" "+str(Dict[int(emp[2])]))

EmployeeNo | Name | Salary | Average Salary of Department
10019 Vito Corleone 170500 59967.79
10015 Jason Foss 178000 97064.64
10272 Debra Houlihan 180000 68684.06
10288 Peter Monroe 157000 97064.64
10086 Katie Roper 150290 97064.64
10222 Ricardo Ruiz 148999 97064.64
10010 Jennifer Zamora 220450 97064.64


3. Compute state wise count of employees from employee.csv

In [ ]:
data = sc.textFile("/content/gdrive/My Drive/employee.csv")

employee = data.map(lambda data: data.split(','))

employee_state = employee.map(lambda rec: (rec[4] , 1))

total_employees_per_state = employee_state.reduceByKey(lambda emp1,emp2 : emp1 + emp2)

print("State | Total Employees")
for value in total_employees_per_state.collect():
  print(value)

State | Total Employees
('MA', 276)
('TX', 3)
('CT', 6)
('VA', 1)
('VT', 2)
('AL', 1)
('WA', 1)
('CA', 1)
('OH', 1)
('IN', 1)
('TN', 1)
('NH', 1)
('RI', 1)
('PA', 1)
('CO', 1)
('NY', 1)
('UT', 1)
('GA', 1)
('FL', 1)
('NC', 1)
('KY', 1)
('ID', 1)
('NV', 1)
('MT', 1)
('OR', 1)
('ND', 1)
('AZ', 1)
('ME', 1)


4. Computing the Standard Deviation of the salary

In [ ]:
data = sc.textFile("/content/gdrive/My Drive/employee.csv")

employee = data.map(lambda data: data.split(','))
employee_salary = employee.map(lambda record: (None, (int(record[3]),1)))

total_employee_salary = employee_salary.reduceByKey(lambda E1 , E2 : (E1[0] + E2[0] ,E1[1]+E2[1]))
total_salary = total_employee_salary.collect()[0][1][0]
total_employee = total_employee_salary.collect()[0][1][1]

def Standard_Deviation(x): #defining a function to calculate the standard deviation
  return (1 , int((int(x) - (total_salary/total_employee))**2))

employee_StandardDeviation = employee.map(lambda record: Standard_Deviation(record[3]))
employee_StandardDeviation_Sum = employee_StandardDeviation.reduceByKey(lambda ST1,ST2 : ST1 + ST2)

total_StandardDeviation = employee_StandardDeviation_Sum.collect()[0][1]
total_StandardDeviation /= total_employee
total_StandardDeviation = math.sqrt(total_StandardDeviation)

#printing the value at the end
print("The Standard Deviation of all salaries ois : "+ str(total_StandardDeviation))

The Standard Deviation of all salaries ois : 25116.159601255837


5. Compute the Department wise Standard Deviation of Salary

In [ ]:
data = sc.textFile("/content/gdrive/My Drive/employee.csv")


employee = data.map(lambda data: data.split(','))
employee_salary = employee.map(lambda record: (None, (int(record[3]),1)))

total_employee_salary = employee_salary.reduceByKey(lambda E1 , E2 : (E1[0] + E2[0] ,E1[1]+E2[1]))

total_salary = total_employee_salary.collect()[0][1][0]
total_employee = total_employee_salary.collect()[0][1][1]


def StandardDeviation(x):
  return (None , int((int(x) - (total_salary/total_employee))**2))

employee_StandardD = employee.map(lambda record: StandardDeviation(record[3]))

employee_StandardD_sum = employee_StandardD.reduceByKey(lambda ST1,ST2 : ST1 + ST2)

total_StandardD = employee_StandardD_sum.collect()[0][1]
total_StandardD = total_StandardD/total_employee
total_StandardD = math.sqrt(total_StandardD)


department_salary = employee.map(lambda record: (record[2], (int(record[3]),1)))

total_department_salary = department_salary.reduceByKey(lambda E1 , E2 : (E1[0] + E2[0] ,E1[1]+E2[1]))

departmentNo_Salary = {}
departmentNo_Total = {}


for i in total_department_salary.collect():
  departmentNo_Salary[int(i[0])] = i[1][0]
  departmentNo_Total[int(i[0])] = i[1][1]

def Department_StandardD(record):
  return int((int(record[3]) - (departmentNo_Salary[int(record[2])]/departmentNo_Total[int(record[2])]))**2)

department_StandardD = employee.map(lambda record: (record[2],Department_StandardD(record)))

department_StandardD_Sum = department_StandardD.reduceByKey(lambda st1,st2 : st1 + st2)

for i in department_StandardD_Sum.collect():
  total_StandardD = i[1]
  total_StandardD = total_StandardD/departmentNo_Total[int(i[0])]
  total_StandardD = math.sqrt(total_StandardD)
  print("The Standard Deviation of Department "+i[0]+" is: "+ str(total_StandardD))

The Standard Deviation of Department 5 is: 11420.800758753909
The Standard Deviation of Department 3 is: 32875.838765269546
The Standard Deviation of Department 4 is: 8754.932432634761
The Standard Deviation of Department 1 is: 19722.68484258672
The Standard Deviation of Department 6 is: 20702.734855569204
The Standard Deviation of Department 2 is: 0.0


6. Listing (empno, dno, name, salary, dept_sal_sd) for employee that are having salary > 1.5 times of SD.

In [ ]:
data = sc.textFile("/content/gdrive/My Drive/employee.csv")

employee = data.map(lambda data: data.split(','))

employee_salary = employee.map(lambda record: (None, (int(record[3]),1)))

total_employee_salary = employee_salary.reduceByKey(lambda e1 , e2 : (e1[0] + e2[0] ,e1[1]+e2[1]))

total_salary = total_employee_salary.collect()[0][1][0]
total_employee = total_employee_salary.collect()[0][1][1]


def StandardD(x):
  return (None , int((int(x) - (total_salary/total_employee))**2))

employee_StandardD = employee.map(lambda record: StandardD(record[3]))

employee_StandardD_Sum = employee_StandardD.reduceByKey(lambda st1,st2 : st1 + st2)

total_StandardD = employee_StandardD_Sum.collect()[0][1]
total_StandardD = total_StandardD / total_employee
employee_final_StandardD = math.sqrt(total_StandardD)


department_salary = employee.map(lambda record: (record[2], (int(record[3]),1)))

tot_dep_sal = department_salary.reduceByKey(lambda E1 , E2 : (E1[0] + E2[0] ,E1[1]+E2[1]))

departmentNo_salary = {}
departmentNo_total = {}
for i in tot_dep_sal.collect():
  departmentNo_salary[int(i[0])] = i[1][0]
  departmentNo_total[int(i[0])] = i[1][1]

def depstd(rec):
  return int((int(rec[3]) - (departmentNo_salary[int(rec[2])]/departmentNo_total[int(rec[2])]))**2)

department_StandardD = employee.map(lambda rec: (rec[2],depstd(rec)))

department_StandardD_Sum = department_StandardD.reduceByKey(lambda st1,st2 : st1 + st2)

departement_StandardD = {}
for i in department_StandardD_Sum.collect():
  total_StandardD = i[1]
  total_StandardD = total_StandardD / departmentNo_total[int(i[0])]
  total_StandardD = math.sqrt(total_StandardD)
  departement_StandardD[int(i[0])] = total_StandardD

employee_department_StandardD = employee.filter(lambda rec: float(rec[3]) > (1.5*employee_final_StandardD))

print("empno dno name salary dept_sal_sd")
for i in employee_department_StandardD .collect():
  print(i[0]+" "+i[1]+" "+i[2]+" "+i[3]+" "+str(departement_StandardD[int(i[2])]))


empno dno name salary dept_sal_sd
10026 Wilson K Adinolfi 5 62506 11420.800758753909
10084 Karthikeyan Ait Sidi 3 104437 32875.838765269546
10196 Sarah Akinkuolie 5 64955 11420.800758753909
10088 Trina Alagbe 5 64991 11420.800758753909
10069 Carol Anderson 5 50825 11420.800758753909
10002 Linda Anderson 5 57568 11420.800758753909
10194 Colby Andreola 4 95660 8754.932432634761
10062 Sam Athwal 5 59365 11420.800758753909
10114 Linda Bachiochi 5 47837 11420.800758753909
10250 Alejandro Bacong 3 50178 32875.838765269546
10252 Rachael Baczenski 5 54670 11420.800758753909
10242 Thomas Barbara 5 47211 11420.800758753909
10012 Hector Barbossa 3 92328 32875.838765269546
10265 Francesco A Barone 5 58709 11420.800758753909
10066 Nader Barton 5 52505 11420.800758753909
10061 Norman Bates 5 57834 11420.800758753909
10023 Kimberly Beak 5 70131 11420.800758753909
10055 Courtney Beatrice 5 59026 11420.800758753909
10245 Renee Becker 3 110000 32875.838765269546
10277 Scott Becker 5 53250 11420.80075875

7. Compute how much offset each department's average salary from the overall average.

In [ ]:
data = sc.textFile("/content/gdrive/My Drive/employee.csv")

employee = data.map(lambda data: data.split(','))

employee_salary = employee.map(lambda rec: (None, (int(rec[3]),1)))

total_employee_salary = employee_salary.reduceByKey(lambda e1 , e2 : (e1[0] + e2[0] ,e1[1]+e2[1]))

total_salary = total_employee_salary.collect()[0][1][0]
total_employee = total_employee_salary.collect()[0][1][1]

overall_average = total_salary/total_employee


department_salary = employee.map(lambda record: (record[2], (int(record[3]),1)))

total_department_salary = department_salary.reduceByKey(lambda E1 , E2 : (E1[0] + E2[0] ,E1[1]+E2[1]))

for i in total_department_salary.collect():
  temp = i[1][0]/i[1][1]
  print("Offset of Department "+i[0]+"'s average salary from the overall average is: "+ str(temp-overall_average))


Offset of Department 5's average salary from the overall average is: -9052.896425921346
Offset of Department 3's average salary from the overall average is: 28043.955112540192
Offset of Department 4's average salary from the overall average is: 27131.4151125402
Offset of Department 1's average salary from the overall average is: 3928.315112540193
Offset of Department 6's average salary from the overall average is: -336.6223874598072
Offset of Department 2's average salary from the overall average is: 180979.3151125402


8. Computes monthly summary on web access log of Lab01, computing:

(a) Total number of requests.
(b) Total download size (in Mega Bytes).

In [ ]:
import re
data = sc.textFile("/content/gdrive/My Drive/web_access_log.txt")

def monthly_summary(record):
  x = re.split('/',record[3])
  x1 = re.split(':',x[2])
  month_of_year = x[1]+" "+x1[0]
  size = 0
  if(record[9]!='-'):
      size = int(record[9])
  return (month_of_year , (1,size))

lines = data.map(lambda data: data.split(' '))

request = lines.map(lambda record: monthly_summary(record))

total_requests_month = request.reduceByKey(lambda r1,r2 : (r1[0]+r2[0] , r1[1]+r2[1]))

for i in total_requests_month.collect():
  print(i[0] + " The total Number of Requests is: "+str(i[1][0])+" and the Download size is : "+str(i[1][1]/(1024**2)))



Dec 2015 The total Number of Requests is: 14148 and the Download size is : 273.58699226379395
Jan 2016 The total Number of Requests is: 28224 and the Download size is : 1755.5749597549438
Feb 2016 The total Number of Requests is: 64262 and the Download size is : 1347.5344944000244


9. List Timestamp , URL of requests in web access for which http response status is 404

In [ ]:
import re
data = sc.textFile("/content/gdrive/My Drive/web_access_log.txt")

lines = data.map(lambda data: data.split(' '))

requests = lines.filter(lambda rec: str(rec[8])=="404") #filterting those with 404 error

count = 1
for i in requests.collect():
  print(str(count)+" "+i[3]+" "+i[6])
  count+=1

1 [12/Dec/2015:19:02:36 /templates/_system/css/general.css
2 [12/Dec/2015:19:44:06 /templates/_system/css/general.css
3 [12/Dec/2015:19:44:15 /favicon.ico
4 [13/Dec/2015:01:01:19 /icons/text.gif
5 [13/Dec/2015:11:28:41 /templates/_system/css/general.css
6 [13/Dec/2015:12:05:25 /apache-log/error.log.44.gz
7 [13/Dec/2015:15:56:36 /apache-log/access.log.69.gz
8 [13/Dec/2015:16:14:58 /apache-log/error.log.55.gz
9 [13/Dec/2015:18:29:14 /templates/_system/css/general.css
10 [13/Dec/2015:18:29:18 /favicon.ico
11 [13/Dec/2015:18:39:08 /templates/_system/css/general.css
12 [13/Dec/2015:20:24:02 /templates/_system/css/general.css
13 [13/Dec/2015:20:27:28 /templates/_system/css/general.css
14 [13/Dec/2015:21:04:08 /templates/_system/css/general.css
15 [13/Dec/2015:21:04:09 /templates/_system/css/general.css
16 [13/Dec/2015:21:04:14 /favicon.ico
17 [13/Dec/2015:21:04:15 /favicon.ico
18 [13/Dec/2015:21:04:48 /templates/_system/css/general.css
19 [13/Dec/2015:21:05:03 /templates/_system/css/general.